<a href="https://colab.research.google.com/github/testgithubprecious/Ml_projects/blob/main/GCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Install required packages if not already:
# pip install torch torchvision torch-geometric torch-scatter torch-sparse torch-cluster torch-spline-conv matplotlib

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
import matplotlib.pyplot as plt

# ------------------------------
# Load dataset
# ------------------------------
dataset = Planetoid(root='data/Cora', name='Cora')
data = dataset[0]  # single graph

# ------------------------------
# Define GCN model
# ------------------------------
class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x

# ------------------------------
# Initialize model & optimizer
# ------------------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(dataset.num_node_features, hidden_channels=16, out_channels=dataset.num_classes).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# ------------------------------
# Training function
# ------------------------------
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

# ------------------------------
# Evaluation function
# ------------------------------
@torch.no_grad()
def test():
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)
    accs = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        acc = pred[mask].eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

# ------------------------------
# Train model
# ------------------------------
for epoch in range(1, 201):
    loss = train()
    train_acc, val_acc, test_acc = test()
    if epoch % 20 == 0:
        print(f"Epoch {epoch:3d} | Loss: {loss:.4f} | Train: {train_acc:.2f} | Val: {val_acc:.2f} | Test: {test_acc:.2f}")